In [1]:
import os 
from glob import glob
import json

import pandas as pd
from data_related.preprocess import *

In [2]:
mapper=pd.read_csv('raw_data/StationCode_mapper.csv',index_col=0)

In [3]:
path_df=pd.DataFrame(glob('raw_data/*.json'),columns=['path'])
path_df=pd.concat([path_df,
                   path_df['path'].apply(lambda x : [x.split('/')[1][:2], x.split('/')[1][3:6]]).apply(pd.Series).rename(columns={0:'type',1:'sc'})], axis=1)

In [4]:
Integrated_data_list=glob('Intergrated_data/*.csv')

In [5]:
# 1. congest data to df
path_df_1=path_df[path_df['type']=='01']

for sc in path_df_1['sc'].unique():
    tmp=path_df_1[path_df_1['sc']==sc]
    tmp_list = list()
    # station_code base data_checker
    tmp_df_info = '_'.join(mapper[mapper['stationCode']==sc].values[0][:-1])
    # if there is data -> pass
    if len([i for i in Integrated_data_list if '01_'+tmp_df_info in i]) != 0:
        continue
    for path in tmp['path']:
        tmp_list.append(json2df(path))
    tmp_df = pd.concat(tmp_list)
    # if congestionTrain == 0 -> not evaluated so drop
    tmp_df=tmp_df[tmp_df['congestionTrain']!=0]
    tmp_df.to_csv(f'Intergrated_data/01_{tmp_df_info}.csv')

In [6]:
# 2. congest_sectional data to df
path_df_2=path_df[path_df['type']=='02']

for sc in path_df_2['sc'].unique():
    tmp=path_df_2[path_df_2['sc']==sc]
    tmp_list = list()
    tmp_df_info = '_'.join(mapper[mapper['stationCode']==sc].values[0][:-1])

    # if there is data -> pass
    if len([i for i in Integrated_data_list if '02_'+tmp_df_info in i]) != 0:
        continue    
    for path in tmp['path']:
        tmp_list.append(json2df(path))
    tmp_df = pd.concat(tmp_list)
#     # if congestionTrain == 0 -> not evaluated so drop
    tmp_df['congestionCar']=tmp_df['congestionCar'].apply(lambda x : x if 0 not in x else None)
    tmp_df.dropna(inplace=True)
    tmp_df_info = '_'.join(mapper[mapper['stationCode']==sc].values[0][:-1])
    tmp_df.to_csv(f'Intergrated_data/02_{tmp_df_info}.csv')

In [7]:
# 3. drop_off data to df
path_df_3=path_df[path_df['type']=='03']

for sc in path_df_3['sc'].unique():
    tmp=path_df_3[path_df_3['sc']==sc]
    tmp_list = list()
    tmp_df_info = '_'.join(mapper[mapper['stationCode']==sc].values[0][:-1])

    # if there is data -> pass
    if len([i for i in Integrated_data_list if '03_'+tmp_df_info in i]) != 0:
        continue    
    for path in tmp['path']:
        tmp_list.append(json2df(path))
    tmp_df = pd.concat(tmp_list)
#     # if congestionTrain == 0 -> not evaluated so drop
    tmp_df['getOffCarRate']=tmp_df['getOffCarRate'].apply(lambda x : x if 0 not in x else None)
    tmp_df.dropna(inplace=True)
    tmp_df.to_csv(f'Intergrated_data/03_{tmp_df_info}.csv')